In [60]:
from os import walk, system
import json
import pandas as pd

In [2]:
from helpers.school import SchoolData

In [3]:
with open('out/2022-09/school_pages_index.json') as fp:
  pages = json.load(fp)

In [4]:
dataframes = []
for id, school_pages in pages.items():
  if len(dataframes) > 500: break
  sd = SchoolData(id, school_pages)
  dataframes.append(sd.computer())

In [5]:
pd.concat(dataframes)

,0,1,2
0,จำนวนคอมพิวเตอร์เพื่อการเรียนการสอน,งบประมาณ สพฐ.,0 เครื่อง
1,จำนวนคอมพิวเตอร์เพื่อการเรียนการสอน,จัดหาเอง/บริจาค:,30 เครื่อง
2,จำนวนคอมพิวเตอร์เพื่อการเรียนการสอน,รวม,30 เครื่อง
3,จำนวนคอมพิวเตอร์เพื่อการเรียนการสอน,ใช้งานได้:,6 เครื่อง
4,จำนวนคอมพิวเตอร์เพื่อการเรียนการสอน,ใช้งานไม่ได้:,24 เครื่อง
...,...,...,...
8,จำนวนคอมพิวเตอร์เพื่อการบริหารจัดการ,ใช้งานได้:,4 เครื่อง
9,จำนวนคอมพิวเตอร์เพื่อการบริหารจัดการ,ใช้งานไม่ได้:,0 เครื่อง
10,จำนวนคอมพิวเตอรทั้งหมด,คอมพิวเตอร์:,47 เครื่อง
11,จำนวนคอมพิวเตอรทั้งหมด,ใช้งานได้:,30 เครื่อง


In [28]:
fpath = school_pages['computer_internet']
tables = pd.read_html(fpath)
for tab in tables:
    bool_tab = ('จำนวนคอมพิวเตอร์เพื่อการเรียนการสอน' == tab)
    bool_tab.any().any()
    if bool_tab.any().any():
        df = tab
        break
df

False
False
False
False
False
True


,0,1
0,จำนวนคอมพิวเตอร์เพื่อการเรียนการสอน,จำนวนคอมพิวเตอร์เพื่อการเรียนการสอน
1,งบประมาณ สพฐ.,0 เครื่อง
2,จัดหาเอง/บริจาค:,20 เครื่อง
3,รวม,20 เครื่อง
4,ใช้งานได้:,0 เครื่อง
5,ใช้งานไม่ได้:,20 เครื่อง
6,จำนวนคอมพิวเตอร์เพื่อการบริหารจัดการ,จำนวนคอมพิวเตอร์เพื่อการบริหารจัดการ
7,งบประมาณ สพฐ.,0 เครื่อง
8,จัดหาเอง/บริจาค:,6 เครื่อง
9,รวม,6 เครื่อง


In [16]:

tables = pd.read_html(school_pages['computer_internet'])
for table in tables:
  if any('จำนวนคอมพิวเตอร์เพื่อการเรียนการสอน' == table):
    df = table

In [17]:
df

,0,1
0,ระบบเครือข่ายอินเทอร์เน็ตที่โรงเรียนเช่าเอง,ระบบเครือข่ายอินเทอร์เน็ตที่โรงเรียนเช่าเอง
1,ผู้ให้บริการ:,-
2,ประเภท:,-
3,ความเร็ว:,-
4,สถานะการใช้งาน:,-
5,งบประมาณ/เดือน:,-
6,ระบบเครือข่าย,ระบบเครือข่าย
7,ระบบเครือข่าย (LAN):,มีระบบเครือข่าย LAN
8,ระบบเครือข่ายไร้สาย (Wireless LAN):,มีระบบเครือข่ายไร้สาย (Wireless LAN)
9,ข้อมูล ณ วันที่ 2021-09-03 18:39:38 น.,ข้อมูล ณ วันที่ 2021-09-03 18:39:38 น.


In [4]:
pd.read_csv('out/2022-09/school_data/internet.csv')

,Unnamed: 0,Unnamed: 0.1,0,1,2,3,school_id,0.1,1.1,2.1,3.1
0,0,0.0,NaN,ยินดีต้อนรับ,NaN,หน้าหลักเขตพื้นที่ | admin page,1010720019,NaN,NaN,NaN,NaN
1,1,NaN,NaN,ยินดีต้อนรับ,NaN,หน้าหลักเขตพื้นที่ | admin page,1010720007,NaN,NaN,NaN,NaN
2,2,NaN,NaN,ยินดีต้อนรับ,NaN,หน้าหลักเขตพื้นที่ | admin page,1010720023,NaN,NaN,NaN,NaN
3,3,NaN,NaN,ยินดีต้อนรับ,NaN,หน้าหลักเขตพื้นที่ | admin page,1010720003,NaN,NaN,NaN,NaN
4,4,NaN,NaN,ยินดีต้อนรับ,NaN,หน้าหลักเขตพื้นที่ | admin page,1010720004,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
29354,29354,NaN,NaN,NaN,NaN,NaN,1018090200,NaN,ยินดีต้อนรับ,NaN,หน้าหลักเขตพื้นที่ | admin page
29355,29355,NaN,NaN,NaN,NaN,NaN,1018090198,NaN,ยินดีต้อนรับ,NaN,หน้าหลักเขตพื้นที่ | admin page
29356,29356,NaN,NaN,NaN,NaN,NaN,1018090201,NaN,ยินดีต้อนรับ,NaN,หน้าหลักเขตพื้นที่ | admin page
29357,29357,NaN,NaN,NaN,NaN,NaN,1018090205,NaN,ยินดีต้อนรับ,NaN,หน้าหลักเขตพื้นที่ | admin page


# Computer and internet

In [2]:
import pandas as pd

In [6]:
dfs = pd.read_html('out/2022/html/school/computer_internet/1010720040.html')

def _df_to_dict(df):
    header = ''
    rows = dict()
    df
    for i, row in df.iterrows():
        if len(row.unique()) != len(row):
            header = row[0]
        else:
            row_list = row.values.tolist()
            col, cell = row_list
            if col not in rows.keys():
                rows[col] = dict()
            rows[col][header] = cell
    return rows
rows = _df_to_dict(dfs[5])
pd.DataFrame(rows)

,index,งบประมาณ สพฐ.,จัดหาเอง/บริจาค:,รวม,ใช้งานได้:,ใช้งานไม่ได้:,คอมพิวเตอร์:
0,จำนวนคอมพิวเตอร์เพื่อการเรียนการสอน,245 เครื่อง,111 เครื่อง,356 เครื่อง,356 เครื่อง,0 เครื่อง,NaN
1,จำนวนคอมพิวเตอร์เพื่อการบริหารจัดการ,0 เครื่อง,0 เครื่อง,0 เครื่อง,0 เครื่อง,0 เครื่อง,NaN
2,จำนวนคอมพิวเตอรทั้งหมด,NaN,NaN,NaN,356 เครื่อง,0 เครื่อง,356 เครื่อง


In [40]:
def _internet_helper(df):
    header = ''
    rows = dict()
    for i, row in df.iterrows():
        if len(row.unique()) != len(row):
            header = row[0]
        else:
            row_list = row.values.tolist()
            col, cell = row_list
            rows[header+'_'+col] = cell
    df = pd.DataFrame([rows.values()], columns=rows.keys())
    return df


,ระบบเครือข่ายอินเทอร์เน็ตที่โรงเรียนเช่าเอง.ผู้ให้บริการ:,ระบบเครือข่ายอินเทอร์เน็ตที่โรงเรียนเช่าเอง.ประเภท:,ระบบเครือข่ายอินเทอร์เน็ตที่โรงเรียนเช่าเอง.ความเร็ว:,ระบบเครือข่ายอินเทอร์เน็ตที่โรงเรียนเช่าเอง.สถานะการใช้งาน:,ระบบเครือข่ายอินเทอร์เน็ตที่โรงเรียนเช่าเอง.งบประมาณ/เดือน:,ระบบเครือข่าย.ระบบเครือข่าย (LAN):,ระบบเครือข่าย.ระบบเครือข่ายไร้สาย (Wireless LAN):
0,3BB,fiber optic,100/100 Mbps,OK Mbps,5000,มีระบบเครือข่าย LAN,มีระบบเครือข่ายไร้สาย (Wireless LAN)


# Durable Goods

In [7]:
from src.indexer import Index

In [36]:
index = Index('out/2022/result_index.txt')

In [37]:
def read_first_line(fpath):
  with open(fpath, 'r') as file:
    line = ''
    c = ''
    while c != '\n':
      c = file.read(1)
      line += c
    return line

In [38]:
%time
read_first_line('out/2022/url_index.txt')

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 16 µs


'https://bobec.bopp-obec.info/build_sch_list.php?Area_CODE=100001\tout/2022/html/area/100001.html\n'

In [39]:
durable_goods = [x for x in index.index.keys() if x.split('.')[1] == 'durable_goods']
headers = {x: read_first_line(index[x]) for x in durable_goods}

In [40]:
def uniq(items):
  unique_items = []
  for i in items:
    if i not in unique_items:
      unique_items.append(i)
  return unique_items

uniq(headers.values())

['ลำดับ,รหัส,รายการ,รายการ(cm.),จำนวนที่ใช้ได้,จำนวนรอซ่อม,จำนวนรอจำหน่าย\n']

In [33]:
target = '<!-- .style37 {color: #FFFFFF; font-family: Tahoma; } .style40 {font-family: Tahoma; font-size: 12px; } -->  เมนูหลัก  หน้าหลักเขตพื้นที่  ข้อมูลพื้นฐานโรงเรียน  ข้อมูลนักเรียน  ข้อมูลครูบุคลากร  ข้อมูลคอมพิวเตอร์ อินเทอร์เน็ต  ข้อมูลสิ่งก่อสร้าง  ข้อมูลครุภัณฑ์  เขตพื้นที่บริการ  ข่าวประชาสัมพันธ์,ข้อมูลครุภัณฑ์ (M-OBEC) โรงเรียนราชสีมาวิทยาลัย\n'

for k, x in headers.items():
  if x == target:
    break

In [35]:
index[k]

'out/2022/school_data/school/1030201355/durable_goods.csv'

# Rename header

In [58]:
filepaths = [
  'out/2022/school_data/building.csv',
  'out/2022/school_data/computer.csv',
  'out/2022/school_data/durable_goods.csv',
  'out/2022/school_data/general.csv',
  'out/2022/school_data/internet.csv',
  'out/2022/school_data/staff.csv',
  'out/2022/school_data/student.csv'
]

In [56]:
# headers = [read_first_line(fp) for fp in filepaths]

In [80]:

with open('header_map.json') as fp:
  mapper = json.load(fp)

In [70]:
dir = 'building'
df = pd.read_csv(f'out/2022/school_data/{dir}.csv', index_col=0)

In [87]:
dir_mapper = mapper[dir]
dir_mapper = {old: new for old, new in dir_mapper.items() if new}
df[['school_id'] + [col for col in df.columns if col != 'school_id']]\
  .rename(dir_mapper, axis=1)\
  .fillna('-')

,school_id,name,model,room_number,build_at,current_condition,image_url_0,image_url_1
0,1010720019,อาคารเรียน ลำดับที่ 1,004,18 ห้อง,พ.ศ. 2497,ดี,https://bobec.bopp-obec.info/b01/7200191012124...,https://bobec.bopp-obec.info/b01/7200191012124...
1,1010720019,อาคารเรียน ลำดับที่ 2,"108 ล,108 ,ล/30,108 หรือ108 ชื่ออื่นๆ",15 ห้อง,พ.ศ. 2528,ดี,https://bobec.bopp-obec.info/b01/7200192012125...,https://bobec.bopp-obec.info/b01/7200192012125...
2,1010720019,บ้านพักภารโรง ลำดับที่ 1,อื่น อื่น,-,พ.ศ. 2538,ดี,https://bobec.bopp-obec.info/b04/7200191040225...,https://bobec.bopp-obec.info/b04/7200191040225...
3,1010720007,อาคารเรียน ลำดับที่ 1,"อื่นๆ 004,008,030,107,206,210,214,315,415,308",24 ห้อง,พ.ศ. 2516,พอใช้,https://bobec.bopp-obec.info/b01/7200071012125...,https://bobec.bopp-obec.info/b01/7200071012125...
4,1010720007,อาคารเรียน ลำดับที่ 2,"อื่นๆ 004,008,030,107,206,210,214,315,415,308",24 ห้อง,พ.ศ. 2529,พอใช้,https://bobec.bopp-obec.info/b01/7200072012125...,https://bobec.bopp-obec.info/b01/7200072012125...
...,...,...,...,...,...,...,...,...
402708,1018090206,อาคารอเนกประสงค์/โรงอาหาร/โรงฝึกงาน/หอประชุม ล...,สปช.202/26,-,พ.ศ. 2530,ดี,https://bobec.bopp-obec.info/b02/9020610202253...,https://bobec.bopp-obec.info/b02/9020610202253...
402709,1018090206,บ้านพักครู ลำดับที่ 1,กรมสามัญ,-,พ.ศ. 2516,พอใช้,https://bobec.bopp-obec.info/b03/9020610304251...,https://bobec.bopp-obec.info/b03/9020610304251...
402710,1018090206,บ้านพักครู ลำดับที่ 2,บ้านพักครู 203/27,-,พ.ศ. 2561,ดี,https://bobec.bopp-obec.info/b03/1801020520309...,https://bobec.bopp-obec.info/b03/1801020520309...
402711,1018090206,ส้วม ลำดับที่ 1,อาคาร สพฐ. 4(ส้วม 4 ที่นั่ง),-,พ.ศ. 2560,พอใช้,https://bobec.bopp-obec.info/b05/1801020560514...,https://bobec.bopp-obec.info/b05/1801020560514...


['school_id',
 'name',
 'แบบ',
 'จำนวนห้องตามแปลน',
 'สร้าง',
 'สภาพปัจจุบัน',
 'image_url_0',
 'image_url_1']

In [75]:
map[dir]

{'school_id': '',
 'name': '',
 'แบบ': 'model',
 'จำนวนห้องตามแปลน': 'room_number',
 'สร้าง': 'build_at',
 'สภาพปัจจุบัน': 'current_condition',
 'image_url_0': '',
 'image_url_1': ''}